In [49]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel, GPT2Config
from transformers import AutoModelForCausalLM, AutoTokenizer
import tqdm
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tutils
import pandas as pd
import torch

In [59]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"USING {device}")

USING cuda


In [60]:
model = GPT2LMHeadModel.from_pretrained("gpt2")#, output_hidden_states=True)
model.config.pad_token_id = model.config.eos_token_id
model.to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


C:\Users\Julia\Documents\LearningC\ChatBot\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [61]:
data = pd.read_csv("data/movie_conversations_data_filtered.csv")
data.dropna(inplace=True)

In [62]:
train_df, test_df = train_test_split(data, test_size=0.5, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42) 

In [63]:
print(len(train_df))
print(len(val_df))

81811
9091


In [64]:
class PTDataset(tutils.Dataset):
  def __init__(self, data: pd.DataFrame):
    self.data = data
    self.max_len = self.logest_seq_len()

  def __len__(self):
    return len(self.data)

  def logest_seq_len(self):
    lens = self.data['Statement'].str.len() + self.data['Reply'].str.len()
    index = lens.idxmax()
    max_len = self.data.loc[index].apply(len).max()
    return max_len


  def __getitem__(self, idx: int):
    row = self.data.iloc[idx]
    statement = row['Statement']
    reply = row['Reply']
    tokenized_statement = tokenizer(statement, padding="max_length", max_length=48, truncation=True, return_tensors="pt")
    tokenized_reply = tokenizer(reply, padding="max_length", max_length=48, truncation=True, return_tensors="pt")
    output_ids = tokenized_reply['input_ids'][tokenized_reply['attention_mask'] == 0] = -100
    return {'input_ids': tokenized_statement['input_ids'][0],
            'attention_mask': tokenized_statement['attention_mask'][0],
            'output_ids': tokenized_reply['input_ids'][0]}

In [65]:
train_data_pt = PTDataset(train_df)
test_data_pt = PTDataset(test_df)
val_data_pt = PTDataset(val_df)

train_loader = tutils.DataLoader(train_data_pt, batch_size=8, shuffle=True)
test_loader = tutils.DataLoader(test_data_pt, batch_size=8)
val_loader = tutils.DataLoader(val_data_pt, batch_size=8, shuffle=False)

In [66]:
UNFREEZE_LAST_N = 3

for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):
    #Only un-freeze the last n transformer blocks
    if i+1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True

for parameter in model.transformer.ln_f.parameters():
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():
    parameter.requires_grad = True

In [67]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

train_losses = []
val_looses = []
scaler = torch.amp.GradScaler('cuda')
for i in range(5):
  running_t_loss = 0
  print(f"EPOCH {i+1}")
  model.train()
  running_loss = 0
  predictions = []
  actual = []
  last_loss = 0
  
  for batch in tqdm.tqdm(train_loader):
    for k, v in batch.items():
      batch[k] = v.to(device)
    input_ids = batch['input_ids']
    output_ids = batch['output_ids']
    mask = batch['attention_mask']
    

    with torch.autocast(device_type=device, dtype=torch.float16):
      output = model(input_ids=input_ids, attention_mask=mask, labels=output_ids)
      loss = output.loss
      logits = output.logits

    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()

    running_t_loss += loss.item()

  av_epoch_loss = running_t_loss / len(train_loader)
  print("Training loss: {:.2f}".format(av_epoch_loss))
  train_losses.append(av_epoch_loss)

  model.eval()
  running_v_loss = 0
  with torch.no_grad():
    for batch in tqdm.tqdm(val_loader):
      for k, v in batch.items():
        batch[k] = v.to(device)
      input_ids = batch['input_ids']
      output_ids = batch['output_ids']
      mask = batch['attention_mask']

      with torch.autocast(device_type=device, dtype=torch.float16):  
        output = model(input_ids=input_ids, attention_mask=mask, labels=output_ids)
        loss = output.loss
        running_v_loss += loss.item()

    av_epoch_loss = running_v_loss / len(val_loader)
    print("Average validation loss: {:.2f}".format(av_epoch_loss))
    val_looses.append(av_epoch_loss)

    print("\n")





EPOCH 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10227/10227 [56:37<00:00,  3.01it/s]


Training loss: 6.52


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1137/1137 [00:56<00:00, 20.12it/s]


Average validation loss: 6.42


EPOCH 2


...

Average validation loss: 6.38


EPOCH 5


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10227/10227 [55:32<00:00,  3.07it/s]


Training loss: 6.24


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1137/1137 [00:57<00:00, 19.65it/s]

Average validation loss: 6.37




In [68]:
torch.backends.cuda.flash_sdp_enabled()

True

In [69]:
model_bare = GPT2LMHeadModel.from_pretrained("gpt2")#, output_hidden_states=True)
model_bare.config.pad_token_id = model_bare.config.eos_token_id
model_bare.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [70]:
model.eval()
text = "No I guess it's okay."
tokenized = tokenizer(text, return_tensors="pt").input_ids.to(device)
#tokenizer(text, padding='max_length', max_length=256, truncation=True, return_tensors="pt")
output = model.generate(tokenized, max_length=50)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [71]:
tokenizer.batch_decode(output)[0]


"No I guess it's okay............................................"

In [67]:
reply_input[reply_mask == 0] = -100
        

In [55]:
train_df.iloc[3]['Statement']
train_df.iloc[3]['Reply']

"No I guess it's okay."

In [102]:
new_id = logits[:, -1, :].argmax(dim=1)  # Generate new ID
input_ids = torch.cat([inputs['input_ids'], new_id.unsqueeze(0)], dim=1)
print(tokenizer.batch_decode(input_ids)[0])

Hello, my dog is cute. I love her. I love


In [100]:
inputs['input_ids'].shape

torch.Size([1, 12])

In [33]:
mask

tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])

In [37]:
train_df.iloc[0]

Statement                                Is that all?
Reply        I've got enough on my mind without that.
Name: 136621, dtype: object

In [128]:
tokenized_statement = tokenizer("Is that all?", padding='max_length', max_length=1025, return_tensors="pt")

In [61]:
train_df.iloc[1]

Statement                               And a snake-eye glare.
Reply        ---and a snake-eye glare Arnold was able to to...
Name: 167275, dtype: object

In [123]:
tokenized_statement['input_ids'].shape

torch.Size([1, 256])